In [1]:
%load_ext autoreload
%autoreload 2
import sys
import platform
from pathlib import Path
from os import environ
if platform.system() == "Darwin": # Nat laptop
    sys.path.extend(['/Users/nkinsky/Documents/UM/GitHub/NeuroPy'])
    # sys.path.extend(['/Users/nkinsky/Documents/UM/GitHub/sleep_deprivation/DataPaths/'])
    sys.path.extend(['/Users/nkinsky/Documents/UM/GitHub/sd_paper/analysis_and_figure_code/DataPaths'])
    plot_dir = Path("/Users/nkinsky/Documents/UM/Working/Bapun_Data/plots")
else:
    if environ["HOSTNAME"] == "lnx00004": # Nat Linux computer
        sys.path.extend(['/data/GitHub/NeuroPy'])
        sys.path.extend(['/data/GitHub/sleep_loss_hippocampal_replay/analysis_and_figure_code/DataPaths'])
        plot_dir = Path('/data/Clustering/plots')
    else:  #BG computer
        sys.path.extend(['/home/nkinsky/Documents/GitHub/NeuroPy'])
        sys.path.extend(['/home/nkinsky/Documents/GitHubPrivate/pythonprogs/DataPaths/'])

group_data_dir = Path('/data/Clustering/grpdata')
    
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import scipy.signal as sg
import matplotlib.pyplot as plt
from neuropy.utils.mathutil import min_max_scaler
from scipy.ndimage import gaussian_filter1d, gaussian_filter
from copy import deepcopy
from tqdm import tqdm

from neuropy import plotting
from neuropy.analyses.placefields import Pf1D, Pf1Dsplit
from neuropy.analyses.oscillations import detect_theta_epochs
from neuropy.core.position import Position
from neuropy.core.epoch import Epoch
from neuropy.plotting.ratemaps import plot_ratemap
from neuropy.plotting.figure import Fig
from neuropy.utils.misc import flatten
from neuropy.io import BinarysignalIO

import subjects
from plotters import Plotter, get_nsd_vs_sd_df, add_parametric_extras, stats_to_df, filter_stats_df
from stats_utils import get_bootstrap_prob, get_bootstrap_prob_paired

plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

/home/nkinsky/miniforge3/envs/NeuroPy/lib/python3.11/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.3, the latest is 0.5.5.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
%matplotlib qt
fig = plt.figure(figsize=(10, 9))
ax1 = fig.add_subplot(311)
ax1.plot([0, 1])
ax2 = fig.add_subplot(312, sharex=ax1)
ax3 = fig.add_subplot(313)

Qt: Session management error: None of the authentication protocols specified are supported


In [3]:
# Create group data frame with placefield firing in each bin (0:50 = MAZE, 50:100 = reMAZE)
from copy import deepcopy
from neuropy.utils.mathutil import min_max_external_scaler

sessions = subjects.remaze_sess()[1:]  # RatSDay2NSD does not have remaze position info
kw = dict(frate_thresh=0, grid_bin=2)
separate_run_dir = False

tc_all, maze_all, remaze_all = [], [], []
pfcorr_by_dir = {}
for s, sess in enumerate([sessions[0]]):
    maze = sess.paradigm["maze"].flatten()
    remaze = sess.paradigm["re-maze"].flatten()
    neurons = sess.neurons_stable.get_neuron_type("pyr")
    
    # ---- Run direction separated-------
    if separate_run_dir:
        tc_maze, tc_remaze, maze_tc_comb, remaze_tc_comb = [], [], [], []
        for direction in ["up", "down"]:
            maze_run = sess.maze_run[direction]
            pfmaze = Pf1D(neurons, position=sess.maze, epochs=maze_run, **kw)
            maze_tc_comb.append(pfmaze.tuning_curves)
            tc_maze.append(pfmaze.resample_1D(50).tuning_curves)
    
            remaze_run = sess.remaze_run[direction]
            pfremaze = Pf1D(neurons, position=sess.remaze, epochs=remaze_run, **kw)
            remaze_tc_comb.append(pfremaze.tuning_curves)
            tc_remaze.append(pfremaze.resample_1D(50).tuning_curves)
    
        tc_comb = np.hstack((np.vstack(tc_maze), np.vstack(tc_remaze)))
        pfmaze.tuning_curves = np.concatenate(maze_tc_comb, axis=0)
        pfmaze.neuron_ids = np.concatenate((pfmaze.neuron_ids, pfmaze.neuron_ids * -1), axis=0)
        pfremaze.tuning_curves = np.concatenate(remaze_tc_comb, axis=0)
        pfremaze.neuron_ids = np.concatenate((pfremaze.neuron_ids, pfremaze.neuron_ids * -1), axis=0)
        

    # ---- Overall----
    else:
        
        pfmaze = Pf1D(neurons, position=sess.maze, **kw)
        pfremaze=Pf1D(neurons, position=sess.remaze, **kw)

In [4]:
from neuropy.analyses.decoders import Decode1d

decode_maze = Decode1d(neurons=neurons, ratemap=pfremaze, epochs=sess.paradigm['re-maze'], bin_size=0.5)

In [5]:
# Simple way to plot
%matplotlib widget
_, ax = plt.subplots(figsize=(10, 3))
speed_thresh = pfremaze.speed > pfremaze.speed_thresh
ax.plot(sess.remaze.time[speed_thresh], sess.remaze.x[speed_thresh], 'b.')
t_decode = sess.paradigm['re-maze'].starts[0] + np.arange(decode_maze.nbins_epochs[0]) * decode_maze.bin_size
ax.plot(t_decode, decode_maze.decoded_position[0], 'r.')

# ax.set_xlim((9900, 10100))

In [6]:
# One way to plot
_, ax = plt.subplots(figsize=(10, 3))
sns.heatmap(decode_maze.posterior[0], cmap="rocket_r", ax=ax, cbar=False)

<Axes: >

In [7]:
# Best way to plot
%matplotlib qt
_, ax = plt.subplots(figsize=(10, 3))
ax.pcolormesh(t_decode, pfremaze.coords.squeeze(), decode_maze.posterior[0], cmap="rocket_r")
ax.plot(sess.remaze.time[speed_thresh], sess.remaze.x[speed_thresh], 'b', linewidth=0.25, label="Actual position")
ax.legend()

sns.despine(ax=ax)

/home/nkinsky/miniforge3/envs/NeuroPy/lib/python3.11/site-packages/ipykernel/eventloops.py:128: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  el.exec() if hasattr(el, 'exec') else el.exec_()


In [8]:
def plot_decode_with_pos(maze_in: Pf1D, decode_in: Decode1d, ax=None):
    if ax is None:
        _, ax = plt.subplots(figsize=(10, 3))
    
    t_decode = maze_in.t_start + np.arange(decode_in.nbins_epochs[0]) * decode_in.bin_size
    ax.pcolormesh(t_decode, decode_in.ratemap.coords.squeeze(), decode_in.posterior[0], cmap="rocket_r")
    ax.plot(maze_in.t, maze_in.x, 'b', linewidth=0.25, label="Actual position")
    ax.legend()

    sns.despine(ax=ax)
    
plot_decode_with_pos(pfremaze, decode_maze)

### Now get decoding error for each session

Define useful function

In [10]:
from copy import deepcopy
def get_decoder_error(decode_in: Decode1d, maze_in: Pf1D, speed_thresh=10):
    """Get decoding error"""

    # Get actual time and position info
    maze_t = maze_in.t
    maze_x = maze_in.x
    maze_speed = maze_in.speed

    # Get decoder time and interpolate position and speed
    t_decode = maze_t.min() + np.arange(decode_in.nbins_epochs[0]) * decode_maze.bin_size
    pos_interp = np.interp(t_decode, maze_t, maze_x)
    speed_interp = np.interp(t_decode, maze_t, maze_speed)
    speed_thresh_bool = speed_interp > speed_thresh

    # Calculate error for all points and for moving times only
    error = decode_in.decoded_position[0] - pos_interp
    error_moving = deepcopy(error)
    error_moving[~speed_thresh_bool] = np.nan

    return error, error_moving

err, err_mv = get_decoder_error(decode_maze, pfremaze)
print(np.nanmean(np.abs(err_mv)))
print(np.nanstd(np.abs(err_mv)))

12.85362524847092
24.21044908969399


In [17]:
# Sanity plot checks - are moving epochs being calculated properly?
from neuropy.utils.mathutil import contiguous_regions
pos_interp = np.interp(t_decode, sess.remaze.time, sess.remaze.x)
speed_interp = np.interp(t_decode, sess.remaze.time, sess.remaze.speed)
speed_interp_thresh_bool = speed_interp > 10
error = decode_maze.decoded_position[0] - pos_interp
error_moving = error[speed_interp_thresh_bool]
print(np.nanmean(np.abs(error_moving)))
print(np.nanstd(np.abs(error_moving)))

_, ax = plt.subplots()
ax.plot(t_decode, pos_interp, 'b-')
mv_inds = contiguous_regions(~np.isnan(err_mv))
for (start, stop) in mv_inds:
    ax.axvspan(t_decode[start], t_decode[stop])


13.780680534934975
28.108244522981597


## Plot out posteriors with position overlaid for each session and calc error
No speed thresholding

In [43]:
# Create group data frame with placefield firing in each bin (0:50 = MAZE, 50:100 = reMAZE)
from copy import deepcopy
from neuropy.utils.mathutil import min_max_external_scaler

sessions = subjects.remaze_sess()[1:]  # RatSDay2NSD does not have remaze position info
kw = dict(frate_thresh=0, grid_bin=2)
frate_thresh_post_hoc = 3
sigma2 = 1.5 # additional smoothing for calculating placefield widths and heights, done in bin units, AFTER tuning curve is created
fr_threshby = "self" # "self" or "both" or "maze"
bin_size = 0.5  # seconds
separate_run_dir = False

tc_all, maze_all, remaze_all = [], [], []
decode_err = []
ind = 0
for s, sess in enumerate(sessions):
    
    fig = plt.figure(figsize=(10, 9))
    fig.suptitle(f"{sess.animal.name} {sess.animal.day} {sess.tag}")
    ax1 = fig.add_subplot(311)
    ax2 = fig.add_subplot(312)
    ax3 = fig.add_subplot(313, sharex=ax2)
    maze = sess.paradigm["maze"].flatten()
    remaze = sess.paradigm["re-maze"].flatten()
    neurons = sess.neurons_stable.get_neuron_type("pyr")
        
    pfmaze = Pf1D(neurons, position=sess.maze, **kw)
    pfremaze = Pf1D(neurons, position=sess.remaze, **kw)

    # Get firing rates and keep only neurons with at least one bin above that value
    frmax_maze = pfmaze.smooth_tuning_curves(sigma_bin=sigma2).max(axis=1)
    frmax_remaze = pfremaze.smooth_tuning_curves(sigma_bin=sigma2).max(axis=1)
    assert fr_threshby in ["maze", "remaze", "self"]
    if fr_threshby == "maze":
        fr_bool_maze = frmax_maze > frate_thresh_post_hoc
        fr_bool_remaze = fr_bool_maze
    elif fr_threshby == "remaze":
        fr_bool_remaze = frmax_remaze > frate_thresh_post_hoc
        fr_bool_maze = fr_bool_remaze
    elif fr_threshby == "self":
        fr_bool_maze = frmax_maze > frate_thresh_post_hoc
        fr_bool_remaze = frmax_remaze > frate_thresh_post_hoc
    neuron_inds_maze = np.where(fr_bool_maze)[0]
    neuron_inds_remaze = np.where(fr_bool_remaze)[0]
    pfmaze = pfmaze.neuron_slice(inds=neuron_inds_maze)
    pfremaze = pfremaze.neuron_slice(inds=neuron_inds_remaze)
    
    decode_maze = Decode1d(neurons=neurons.neuron_slice(neuron_inds=neuron_inds_maze), ratemap=pfmaze, 
                           epochs=sess.paradigm['maze'], bin_size=bin_size)
    decode_remaze = Decode1d(neurons=neurons.neuron_slice(neuron_inds=neuron_inds_remaze), ratemap=pfremaze, 
                             epochs=sess.paradigm['re-maze'], bin_size=bin_size)
    decode_maze_remaze = Decode1d(neurons=neurons.neuron_slice(neuron_inds=neuron_inds_maze), ratemap=pfmaze, 
                                  epochs=sess.paradigm['re-maze'], bin_size=bin_size)

    for stype, maze_in, decode_in, ax in zip(["MAZE", "reMAZE", "MAZE-reMAZE"], [pfmaze, pfremaze, pfremaze],
                                        [decode_maze, decode_remaze, decode_maze_remaze], [ax1, ax2, ax3]):
        err, err_mv = get_decoder_error(decode_in, maze_in)
        decode_err.append(pd.DataFrame({"session": sess.name, "grp": sess.tag, "decode_type": stype, 
                                        "error_mean_all": np.nanmean(np.abs(err)), "error_std_all": np.nanstd(np.abs(err)),
                                        "error_mean_move": np.nanmean(np.abs(err_mv)), "error_std_move": np.nanstd(np.abs(err_mv)),
                                        "n_neurons": maze_in.n_neurons
                                       }, 
                                       index=[ind]))
        ind += 1
        plot_decode_with_pos(maze_in, decode_in, ax)

decode_err_df = pd.concat(decode_err, axis=0)

/home/nkinsky/miniforge3/envs/NeuroPy/lib/python3.11/site-packages/ipykernel/eventloops.py:128: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  el.exec() if hasattr(el, 'exec') else el.exec_()


In [44]:
decode_err_df

,session,grp,decode_type,error_mean_all,error_std_all,error_mean_move,error_std_move,n_neurons
0,RatUDay2,NSD,MAZE,35.499304,65.387743,15.606178,24.805398,85
1,RatUDay2,NSD,reMAZE,43.736018,72.288879,12.273756,22.326070,83
2,RatUDay2,NSD,MAZE-reMAZE,60.593842,81.309199,28.518900,45.016957,83
3,RatVDay1,NSD,MAZE,93.529660,93.224522,66.623940,73.764662,16
4,RatVDay1,NSD,reMAZE,88.874529,97.595816,50.867203,71.855924,16
5,RatVDay1,NSD,MAZE-reMAZE,78.577778,95.687485,68.854294,79.892314,16
6,RatVDay3,NSD,MAZE,76.061994,93.407497,62.556514,78.093489,19
7,RatVDay3,NSD,reMAZE,103.908854,101.445975,65.090171,80.495069,17
8,RatVDay3,NSD,MAZE-reMAZE,152.771528,93.809512,137.064902,77.196549,17
9,RatSDay3,SD,MAZE,83.693537,105.359797,42.933514,64.900246,24


#### Sanity check plot - is decoding error related to # neurons?
Yes!

In [45]:
_, ax = plt.subplots()
sns.scatterplot(data=decode_err_df, x="n_neurons", y="error_mean_move", ax=ax)

<Axes: xlabel='n_neurons', ylabel='error_mean_move'>

## Now decode but using speed and firing rate thresholded placefields

In [47]:
kw = dict(frate_thresh=3, grid_bin=5, speed_thresh=10, sigma=0)
sigma2 = 1.5 # additional smoothing for calculating placefield widths, done in bin units, AFTER tuning curve is created
frate_thresh_post_hoc = 3

bin_size = 0.5  # seconds
separate_run_dir = False
nshuf = 1000 # for calculating chance-level decoding

tc_all, maze_all, remaze_all = [], [], []
decode_err, decode_err_shuf = [], []
ind = 0
for s, sess in enumerate(sessions):
    
    fig = plt.figure(figsize=(10, 9))
    fig.suptitle(f"{sess.animal.name} {sess.animal.day} {sess.tag}")
    ax1 = fig.add_subplot(311)
    ax2 = fig.add_subplot(312)
    ax3 = fig.add_subplot(313, sharex=ax2)
    maze = sess.paradigm["maze"].flatten()
    remaze = sess.paradigm["re-maze"].flatten()
    neurons = sess.neurons_stable.get_neuron_type("pyr")
    
    maze_run = sess.maze_run
    run_bool = sess.maze.get_smoothed(0.1).speed > kw["speed_thresh"]
    run_epochs = Epoch.from_boolean_array(run_bool, t=sess.maze.time)
    maze_run_thresh = maze_run.intersection(run_epochs, res=1/sess.maze.sampling_rate)
    pfmaze = Pf1D(neurons, position=sess.maze, epochs=maze_run_thresh, sigma_pos=0.1, **kw)
    neurons_maze = neurons.neuron_slice(neuron_ids=pfmaze.neuron_ids)
    
    remaze_run = sess.remaze_run
    rm_run_bool = sess.remaze.get_smoothed(0.1).speed > kw["speed_thresh"]
    rm_run_epochs = Epoch.from_boolean_array(rm_run_bool, t=sess.remaze.time)
    remaze_run_thresh = remaze_run.intersection(rm_run_epochs, res=1/sess.maze.sampling_rate)
    pfremaze = Pf1D(neurons, position=sess.remaze, epochs=remaze_run_thresh, sigma_pos=0.1, **kw)
    neurons_remaze = neurons.neuron_slice(neuron_ids=pfremaze.neuron_ids)
#     fr_max = np.vstack((pfmaze.tuning_curves.max(axis=1), pfremaze.tuning_curves.max(axis=1))).T
    
    decode_maze = Decode1d(neurons=neurons_maze, ratemap=pfmaze, epochs=sess.paradigm['maze'], bin_size=bin_size)
    decode_remaze = Decode1d(neurons=neurons_remaze, ratemap=pfremaze, epochs=sess.paradigm['re-maze'], bin_size=bin_size)
    decode_maze_remaze = Decode1d(neurons=neurons_maze, ratemap=pfmaze, epochs=sess.paradigm['re-maze'], bin_size=bin_size)

    for stype, maze_in, decode_in, ax in zip(["MAZE", "reMAZE", "MAZE-reMAZE"], [pfmaze, pfremaze, pfremaze],
                                             [decode_maze, decode_remaze, decode_maze_remaze], [ax1, ax2, ax3]):
        err, err_mv = get_decoder_error(decode_in, maze_in, speed_thresh=kw["speed_thresh"])
        
        # now get chance-level decoding
        decode_shuf = deepcopy(decode_in)
        err_mv_mean_shuf, err_all_mean_shuf = [], []
        for i in tqdm(np.arange(nshuf)):
            decode_shuf.ratemap.tuning_curves = decode_shuf.ratemap.tuning_curves[np.random.permutation(decode_shuf.ratemap.n_neurons)]
            decode_shuf._estimate()
            err_shuf, err_mv_shuf = get_decoder_error(decode_shuf, maze_in, speed_thresh=kw["speed_thresh"])
            err_mv_mean_shuf.append(np.nanmean(np.abs(err_mv_shuf)))
            err_all_mean_shuf.append(np.nanmean(np.abs(err_shuf)))
        
        
        decode_err.append(pd.DataFrame({"session": sess.name, "grp": sess.tag, "decode_type": stype, 
                                        "error_mean_all": np.nanmean(np.abs(err)), 
                                        "error_std_all": np.nanstd(np.abs(err)),
                                        "error_mean_move": np.nanmean(np.abs(err_mv)), 
                                        "error_std_move": np.nanstd(np.abs(err_mv)),
                                        "n_neurons": maze_in.n_neurons
                                       }, index=[ind]))
        decode_err_shuf.append(pd.DataFrame({"session": sess.name, "grp": sess.tag, "decode_type": stype,
                                             "error_mean_shuf_move": err_mv_mean_shuf,
                                             "error_mean_shuf_all": err_all_mean_shuf}))
        ind += 1
        plot_decode_with_pos(maze_in, decode_in, ax)

decode_err_thresh_df = pd.concat(decode_err, axis=0)
decode_err_shuf_df = pd.concat(decode_err_shuf, axis=0)

Note: speed_thresh is ignored when epochs is provided
Note: speed_thresh is ignored when epochs is provided


100%|█████████████████████████████████████████| 100/100 [00:28<00:00,  3.57it/s]


Note: speed_thresh is ignored when epochs is provided
Note: speed_thresh is ignored when epochs is provided


100%|█████████████████████████████████████████| 100/100 [00:05<00:00, 19.09it/s]


Note: speed_thresh is ignored when epochs is provided
Note: speed_thresh is ignored when epochs is provided


...

100%|█████████████████████████████████████████| 100/100 [00:16<00:00,  6.20it/s]


Note: speed_thresh is ignored when epochs is provided
Note: speed_thresh is ignored when epochs is provided


100%|█████████████████████████████████████████| 100/100 [00:06<00:00, 14.52it/s]


Note: speed_thresh is ignored when epochs is provided
Note: speed_thresh is ignored when epochs is provided


100%|█████████████████████████████████████████| 100/100 [00:06<00:00, 15.47it/s]
/home/nkinsky/miniforge3/envs/NeuroPy/lib/python3.11/site-packages/ipykernel/eventloops.py:128: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  el.exec() if hasattr(el, 'exec') else el.exec_()


In [48]:
decode_err_shuf_df.to_csv(group_data_dir / "maze_remaze_decode_error_shuffle.csv")
decode_err_thresh_df.to_csv(group_data_dir / "maze_remaze_decode_error_speed_thresh.csv")

In [49]:
decode_err_thresh_df

,session,grp,decode_type,error_mean_all,error_std_all,error_mean_move,error_std_move,n_neurons
0,RatUDay2,NSD,MAZE,48.142452,74.773267,19.880250,32.828011,97
1,RatUDay2,NSD,reMAZE,69.507724,89.062470,19.657794,38.062956,95
2,RatUDay2,NSD,MAZE-reMAZE,97.582121,97.195752,47.915045,64.912137,95
3,RatVDay1,NSD,MAZE,111.904640,100.195420,73.130845,81.262592,16
4,RatVDay1,NSD,reMAZE,91.031471,98.244886,52.645244,71.291037,17
5,RatVDay1,NSD,MAZE-reMAZE,84.192522,100.574972,79.429786,85.309809,17
6,RatVDay3,NSD,MAZE,78.886641,94.565941,59.589712,76.628788,22
7,RatVDay3,NSD,reMAZE,85.875502,90.482370,47.296301,66.610079,21
8,RatVDay3,NSD,MAZE-reMAZE,145.202876,93.620703,129.948769,78.887090,21
9,RatSDay3,SD,MAZE,99.036202,110.379904,48.013724,71.988155,27


In [196]:
_, ax = plt.subplots()
sns.stripplot(data=decode_err_thresh_df, x="grp", y="error_mean_move", hue="decode_type", dodge=True, ax=ax)
sns.boxplot(data=decode_err_shuf_df, x="grp", y="error_mean_shuf_move", hue="decode_type", dodge=True, 
            legend=False, ax=ax)

<Axes: xlabel='grp', ylabel='error_mean_move'>

In [197]:
_, ax = plt.subplots()
sns.stripplot(data=decode_err_thresh_df, x="grp", y="error_std_move", hue="decode_type", dodge=True, ax=ax)

<Axes: xlabel='grp', ylabel='error_std_move'>

In [ ]:
# Best way to plot
_, ax = plt.subplots(figsize=(10, 3))
t_decode_maze_remaze = sess.paradigm['re-maze'].starts[0] + np.arange(decode_maze_remaze.nbins_epochs[0]) * decode_maze_remaze.bin_size
ax.pcolormesh(t_decode, pfmaze.coords.squeeze(), decode_maze_remaze.posterior[0], cmap="rocket_r")
ax.plot(sess.remaze.time[speed_thresh], sess.remaze.x[speed_thresh], 'b', linewidth=0.25, label="Actual position")
ax.legend()

sns.despine(ax=ax)